In [99]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]")\
          .appName("sunshine_v2")\
          .getOrCreate()

In [100]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import *

In [101]:
individu = spark.read.\
       option("header", "true").\
       option("delimiter", ";").\
       csv("/Users/youssouf/Documents/simboxv2/df_echantillon_train_simbox.csv")

individu = individu.dropDuplicates()
individu = individu.toDF(*(c.replace('.', '_') for c in individu.columns))
individu.show()

+-------------------------------+-------------------------------+
|echantillon_train_simbox_msisdn|echantillon_train_simbox_statut|
+-------------------------------+-------------------------------+
|                       09664314|                              0|
|                       59396661|                              0|
|                       08426232|                              0|
|                       07736571|                              0|
|                       08040534|                              0|
|                       68920076|                              0|
|                       77121885|                              0|
|                       97228552|                              0|
|                       88384131|                              0|
|                       88620304|                              0|
|                       88163609|                              0|
|                       57849472|                              0|
|         

### Nombre de fraudeurs et non fraudeurs

In [84]:
individu.groupby("echantillon_train_simbox_statut").count().show()

+-------------------------------+-------+
|echantillon_train_simbox_statut|  count|
+-------------------------------+-------+
|                              0|1016375|
|                              1|   1960|
+-------------------------------+-------+



In [86]:
(1960 / 1016375)*100

0.19284220883040215

In [ ]:
#simbox_temp_cons_sms_msisdn

In [102]:
temp_cons_call = spark.read.\
       option("header", "true").\
       option("delimiter", ";").\
       csv("/Users/youssouf/Documents/simboxv2/df_sunshine_simbox_temp_cons_call.csv")

temp_cons_call = temp_cons_call.dropDuplicates()
temp_cons_call = temp_cons_call.toDF(*(c.replace('.', '_') for c in temp_cons_call.columns))
temp_cons_call.show()

+----------------------------+------------------------------------+----------------------------------------+
|simbox_temp_cons_call_msisdn|simbox_temp_cons_call_temp_appel_moy|simbox_temp_cons_call_ecartyp_temp_appel|
+----------------------------+------------------------------------+----------------------------------------+
|                    07293076|                   5906.605839416058|                       8518.525616370136|
|                    07480843|                  3434.3872549019607|                       5480.182841163482|
|                    07751948|                             17844.0|                                 10863.0|
|                    08009927|                                NULL|                                    NULL|
|                    08129412|                              6292.0|                       7311.198830105314|
|                    08423262|                  10323.266666666666|                      10045.844507169231|
|                  

In [103]:
temp_cons_sms = spark.read.\
       option("header", "true").\
       option("delimiter", ";").\
       csv("/Users/youssouf/Documents/simboxv2/df_sunshine_simbox_temp_cons_sms.csv")

temp_cons_sms = temp_cons_sms.dropDuplicates()
temp_cons_sms = temp_cons_sms.toDF(*(c.replace('.', '_') for c in temp_cons_sms.columns))
temp_cons_sms.show()

+---------------------------+---------------------------------+-------------------------------------+
|simbox_temp_cons_sms_msisdn|simbox_temp_cons_sms_temp_sms_moy|simbox_temp_cons_sms_ecartyp_temp_sms|
+---------------------------+---------------------------------+-------------------------------------+
|                   07117188|               2025.0854271356784|                    5110.096388841895|
|                   07234808|               2385.6276422764226|                    5706.241091246586|
|                   07396114|                             NULL|                                 NULL|
|                   07410540|                             NULL|                                 NULL|
|                   07958942|                             NULL|                                 NULL|
|                   08369306|                885.1705882352941|                    3024.420066969516|
|                   08546231|                             NULL|                   

In [104]:
imsi = spark.read.\
       option("header", "true").\
       option("delimiter", ";").\
       csv("/Users/youssouf/Documents/simboxv2/df_sunshine_simbox_imsi.csv")

imsi = imsi.dropDuplicates()
imsi = imsi.toDF(*(c.replace('.', '_') for c in imsi.columns))
imsi.show()

+------------------+---------------------+
|simbox_imsi_msisdn|simbox_imsi_nbre_imsi|
+------------------+---------------------+
|          07189240|                    1|
|          07329996|                    1|
|          07559648|                    1|
|          07809004|                    1|
|          08185074|                    1|
|          09111412|                    1|
|          09630349|                    1|
|          48322309|                    1|
|          48550629|                    1|
|          49271746|                    1|
|          57759808|                    1|
|          58625068|                    1|
|          58798460|                    1|
|          58831889|                    1|
|          58842914|                    1|
|          59117921|                    1|
|          59641177|                    1|
|          67063161|                    1|
|          67291704|                    1|
|          68502791|                    1|
+----------

In [105]:
complice = spark.read.\
       option("header", "true").\
       option("delimiter", ";").\
       csv("/Users/youssouf/Documents/simboxv2/df_sunshine_simbox_complice.csv")

complice = complice.dropDuplicates()
complice = complice.toDF(*(c.replace('.', '_') for c in complice.columns))
complice.show()

+-----------------------+----------------------+--------------------------------------+-------------------------------------+---------------------------------------+---------------------------------------+
|simbox_complice_date_id|simbox_complice_msisdn|simbox_complice_nbr_dist_call_complice|simbox_complice_nbr_call_tot_complice|simbox_complice_duree_tot_call_complice|simbox_complice_duree_moy_call_complice|
+-----------------------+----------------------+--------------------------------------+-------------------------------------+---------------------------------------+---------------------------------------+
|               20201021|              07412786|                                     3|                                    5|                                   5417|                                 1083.4|
|                   NULL|              08168853|                                  NULL|                                 NULL|                                   NULL|           

In [106]:
entrant_call = spark.read.\
       option("header", "true").\
       option("delimiter", ";").\
       csv("/Users/youssouf/Documents/simboxv2/df_sunshine_simbox_entrant_call.csv")

entrant_call = entrant_call.dropDuplicates()
entrant_call = entrant_call.toDF(*(c.replace('.', '_') for c in entrant_call.columns))
entrant_call.show()

+---------------------------+--------------------------+------------------------------------+-----------------------------------+-------------------------------------+-------------------------------------+
|simbox_entrant_call_date_id|simbox_entrant_call_msisdn|simbox_entrant_call_nbr_dist_call_en|simbox_entrant_call_nbr_call_tot_en|simbox_entrant_call_duree_call_tot_en|simbox_entrant_call_duree_call_moy_en|
+---------------------------+--------------------------+------------------------------------+-----------------------------------+-------------------------------------+-------------------------------------+
|                   20201024|                  07440520|                                   2|                                  2|                                   65|                                 32.5|
|                       NULL|                  08168853|                                NULL|                               NULL|                                 NULL|         

In [107]:
entrant_sms = spark.read.\
       option("header", "true").\
       option("delimiter", ";").\
       csv("/Users/youssouf/Documents/simboxv2/df_sunshine_simbox_entrant_sms.csv")

entrant_sms = entrant_sms.dropDuplicates()
entrant_sms = entrant_sms.toDF(*(c.replace('.', '_') for c in entrant_sms.columns))
entrant_sms.show()


+--------------------------+-------------------------+----------------------------------+---------------------------------+
|simbox_entrant_sms_date_id|simbox_entrant_sms_msisdn|simbox_entrant_sms_nbr_dist_sms_en|simbox_entrant_sms_nbr_sms_tot_en|
+--------------------------+-------------------------+----------------------------------+---------------------------------+
|                  20201016|                 07555141|                                 5|                               16|
|                  20201016|                 07759767|                                 1|                                2|
|                  20201016|                 07777396|                                 1|                                6|
|                      NULL|                 08638053|                              NULL|                             NULL|
|                      NULL|                 09088524|                              NULL|                             NULL|
|       

In [25]:
#rechargement_om = spark.read.\
#       option("header", "true").\
#       option("delimiter", ";").\
#       csv("/Users/youssouf/Documents/simboxv2/df_sunshine_simbox_rechargement_om.csv")

#rechargement_om.show()
#rechargement_om = rechargement_om.dropDuplicates()

In [98]:
#for cl in liste:
#    df3 = df3.withColumn(cl, col(cl).cast(DoubleType()))
#df3 = df3.fillna(0)

### Jointure des data frames 

In [145]:
df = individu.join(temp_cons_call, individu.echantillon_train_simbox_msisdn == temp_cons_call.simbox_temp_cons_call_msisdn, "left_outer" ).drop("simbox_temp_cons_sms_msisdn") \
             .join(temp_cons_sms, individu.echantillon_train_simbox_msisdn == temp_cons_sms.simbox_temp_cons_sms_msisdn , "left_outer").drop("simbox_temp_cons_sms_msisdn") \
             .join(imsi, individu.echantillon_train_simbox_msisdn == imsi.simbox_imsi_msisdn , "left_outer").drop("simbox_imsi_msisdn") \
             .join(complice, individu.echantillon_train_simbox_msisdn == complice.simbox_complice_msisdn , "left_outer").drop("simbox_complice_msisdn", "simbox_complice_date_id") \
#             .join(entrant_call, individu.echantillon_train_simbox_msisdn == entrant_call.simbox_entrant_call_msisdn , "left_outer").drop("simbox_entrant_call_msisdn","simbox_entrant_call_date_id") \
#             .join(entrant_sms, individu.echantillon_train_simbox_msisdn == entrant_sms.simbox_entrant_sms_msisdn , "left_outer").drop("simbox_entrant_sms_msisdn","simbox_entrant_sms_date_id")



In [146]:
df.columns

['echantillon_train_simbox_msisdn',
 'echantillon_train_simbox_statut',
 'simbox_temp_cons_call_msisdn',
 'simbox_temp_cons_call_temp_appel_moy',
 'simbox_temp_cons_call_ecartyp_temp_appel',
 'simbox_temp_cons_sms_temp_sms_moy',
 'simbox_temp_cons_sms_ecartyp_temp_sms',
 'simbox_imsi_nbre_imsi']

In [147]:
df.groupby("echantillon_train_simbox_statut").count().show()

+-------------------------------+-------+
|echantillon_train_simbox_statut|  count|
+-------------------------------+-------+
|                              0|1016375|
|                              1|   1960|
+-------------------------------+-------+



In [137]:
df.columns

['echantillon_train_simbox_msisdn',
 'echantillon_train_simbox_statut',
 'simbox_temp_cons_call_msisdn',
 'simbox_temp_cons_call_temp_appel_moy',
 'simbox_temp_cons_call_ecartyp_temp_appel',
 'simbox_temp_cons_sms_temp_sms_moy',
 'simbox_temp_cons_sms_ecartyp_temp_sms',
 'simbox_imsi_nbre_imsi']

In [148]:
coll = ['msisdn',
 'statut',
 'xmsisdn',
 'temp_appel_moy',
 'ecartyp_temp_appel',
 'temp_sms_moy',
 'ecartyp_temp_sms',
 'nbre_imsi']

In [149]:
df = df.toDF('msisdn',
 'statut',
 'xmsisdn',
 'temp_appel_moy',
 'ecartyp_temp_appel',
 'temp_sms_moy',
 'ecartyp_temp_sms',
 'nbre_imsi')

In [150]:
df = df.drop("xmsisdn")

### Count Na'Values by Column

In [57]:
from pyspark.sql.functions import isnan, when, count, col

df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

+------+------+------------+----------------+-------------+-----------------+---------+----------------------+---------------------+-----------------------+-----------------------+---------------------+--------------------+----------------------+----------------------+-------------------+------------------+
|msisdn|statut|temp_sms_moy|ecartyp_temp_sms|temp_sms_moy1|ecartyp_temp_sms1|nbre_imsi|nbr_dist_call_complice|nbr_call_tot_complice|duree_tot_call_complice|duree_moy_call_complice|call_nbr_dist_call_en|call_nbr_call_tot_en|call_duree_call_tot_en|call_duree_call_moy_en|sms_nbr_dist_sms_en|sms_nbr_sms_tot_en|
+------+------+------------+----------------+-------------+-----------------+---------+----------------------+---------------------+-----------------------+-----------------------+---------------------+--------------------+----------------------+----------------------+-------------------+------------------+
|     0|     0|           0|               0|            0|              

### Taille du DataFrame

In [140]:
df.count(), len(df.columns)

(1018335, 8)

### Afficahge du Dataframe

In [141]:
df.printSchema()

root
 |-- msisdn: string (nullable = true)
 |-- statut: string (nullable = true)
 |-- msisdn: string (nullable = true)
 |-- temp_appel_moy: string (nullable = true)
 |-- ecartyp_temp_appel: string (nullable = true)
 |-- temp_sms_moy: string (nullable = true)
 |-- ecartyp_temp_sms: string (nullable = true)
 |-- nbre_imsi: string (nullable = true)



In [151]:
df.show()

+--------+------+------------------+------------------+------------------+------------------+---------+
|  msisdn|statut|    temp_appel_moy|ecartyp_temp_appel|      temp_sms_moy|  ecartyp_temp_sms|nbre_imsi|
+--------+------+------------------+------------------+------------------+------------------+---------+
|07000421|     0| 2527.929411764706|3395.9711693788854| 1101.341517857143|4422.6069336384335|        1|
|07001041|     0| 5827.075268817204| 7931.733563157254| 6748.307692307692|10665.039416311563|        1|
|07003963|     0| 1929.031328320802|2941.7575910639625| 2461.407854984894| 5781.348364020444|        1|
|07004359|     0| 4190.748743718593| 6304.263389294098| 3403.548387096774| 8836.558693178089|        1|
|07004719|     0|        5712.53125| 7888.953742751534|20378.333333333332| 13733.95026661761|        1|
|07004893|     0| 5207.203389830508| 7277.673950385803| 2878.076433121019| 6447.661046848671|        1|
|07005380|     0| 5463.145161290323| 7522.883650197714| 2175.622

### Replace NULL , null by 0

In [152]:
df = df.replace('NULL',str(0))
#df = df.replace(null ,str(0))
df = df.na.fill(0)

### Checking if done 

In [153]:
df.show()

+--------+------+------------------+------------------+------------------+------------------+---------+
|  msisdn|statut|    temp_appel_moy|ecartyp_temp_appel|      temp_sms_moy|  ecartyp_temp_sms|nbre_imsi|
+--------+------+------------------+------------------+------------------+------------------+---------+
|07000421|     0| 2527.929411764706|3395.9711693788854| 1101.341517857143|4422.6069336384335|        1|
|07001041|     0| 5827.075268817204| 7931.733563157254| 6748.307692307692|10665.039416311563|        1|
|07003963|     0| 1929.031328320802|2941.7575910639625| 2461.407854984894| 5781.348364020444|        1|
|07004359|     0| 4190.748743718593| 6304.263389294098| 3403.548387096774| 8836.558693178089|        1|
|07004719|     0|        5712.53125| 7888.953742751534|20378.333333333332| 13733.95026661761|        1|
|07004893|     0| 5207.203389830508| 7277.673950385803| 2878.076433121019| 6447.661046848671|        1|
|07005380|     0| 5463.145161290323| 7522.883650197714| 2175.622

### Nombre de fraudeurs

In [154]:
df.groupby("statut").count().show()

+------+-------+
|statut|  count|
+------+-------+
|     0|1016375|
|     1|   1960|
+------+-------+



### Converting to DOUBLE

In [155]:
liste = [
 'statut',
 'temp_appel_moy',
 'ecartyp_temp_appel',
 'temp_sms_moy',
 'ecartyp_temp_sms',
 'nbre_imsi']

for cl in liste:
    df = df.withColumn(cl, col(cl).cast(DoubleType()))

## Models

### Random Forest Model

In [166]:
df = df.withColumnRenamed("statut", "label")

In [167]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator


In [168]:
feature_list = [
 'temp_appel_moy',
 'ecartyp_temp_appel',
 'temp_sms_moy',
 'ecartyp_temp_sms',
 'nbre_imsi']

In [169]:
assembler = VectorAssembler(inputCols=feature_list , outputCol="features")

In [170]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

In [171]:
pipeline = Pipeline(stages=[assembler, rf])

In [177]:
from pyspark.ml.tuning import ParamGridBuilder
import numpy as np
paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [int(x) for x in np.linspace(start = 10, stop = 13, num = 3)]) \
    .addGrid(rf.maxDepth, [int(x) for x in np.linspace(start = 5, stop = 8, num = 3)]) \
    .build()

In [178]:
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=3)

In [179]:
(trainingData, testData) = df.randomSplit([0.8, 0.2])

In [185]:
trainingData.groupby("label").count().show()

+-----+------+
|label| count|
+-----+------+
|  0.0|812441|
|  1.0|  1558|
+-----+------+



In [187]:
def resample(base_features,ratio,class_field,base_class):
    pos = base_features.filter(col(class_field)==base_class)
    neg = base_features.filter(col(class_field)!=base_class)
    total_pos = pos.count()
    total_neg = neg.count()
    fraction=float(total_pos*ratio)/float(total_neg)
    sampled = neg.sample(False,fraction)
    return sampled.union(pos)

In [201]:
trainingData_ = resample(trainingData, 1.5 , "label", 1.0)

In [202]:
trainingData_.groupby("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0| 2340|
|  1.0| 1558|
+-----+-----+



In [186]:
testData.groupby("label").count().show()

+-----+------+
|label| count|
+-----+------+
|  0.0|203934|
|  1.0|   402|
+-----+------+



In [203]:
cvModel = crossval.fit(trainingData_)

In [204]:
predictions = cvModel.transform(testData)

In [182]:
predictions.show(3)

+--------+-----+-----------------+------------------+-----------------+------------------+---------+--------------------+--------------------+--------------------+----------+
|  msisdn|label|   temp_appel_moy|ecartyp_temp_appel|     temp_sms_moy|  ecartyp_temp_sms|nbre_imsi|            features|       rawPrediction|         probability|prediction|
+--------+-----+-----------------+------------------+-----------------+------------------+---------+--------------------+--------------------+--------------------+----------+
|07001041|  0.0|5827.075268817204| 7931.733563157254|6748.307692307692|10665.039416311563|      1.0|[5827.07526881720...|[12.9904156441858...|[0.99926274186044...|       0.0|
|07003963|  0.0|1929.031328320802|2941.7575910639625|2461.407854984894| 5781.348364020444|      1.0|[1929.03132832080...|[12.9528745850120...|[0.99637496807784...|       0.0|
|07004359|  0.0|4190.748743718593| 6304.263389294098|3403.548387096774| 8836.558693178089|      1.0|[4190.74874371859...|[12.

In [205]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()
print('Test Area Under ROC', evaluator.evaluate(predictions))

Test Area Under ROC 0.8917839090170963


In [28]:
#print('Training set areaUnderROC: ' + str(trainingSummary.areaUnderROC))

+-------------------------------+
|echantillon_train_simbox.msisdn|
+-------------------------------+
|                       09664314|
|                       59396661|
|                       08426232|
|                       07736571|
|                       08040534|
|                       68920076|
|                       77121885|
|                       97228552|
|                       88384131|
|                       88620304|
|                       88163609|
|                       57849472|
|                       09965894|
|                       98374630|
|                       88577153|
|                       58458341|
|                       58142139|
|                       08733056|
|                       48743148|
|                       59666862|
+-------------------------------+
only showing top 20 rows



In [214]:
predictions.groupby("prediction").count().show()

+----------+------+
|prediction| count|
+----------+------+
|       0.0|184490|
|       1.0| 19846|
+----------+------+



### MultilayerPerceptron

In [209]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [210]:
# specify layers for the neural network:
# input layer of size 5 (features), two intermediate of size 5 and 4
# and output of size 2 (classes)
layers = [5, 5, 4, 2]



# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)


pipeline = Pipeline(stages=[assembler, trainer])


# train the model
model = pipeline.fit(trainingData_)

In [211]:
# compute accuracy on the test set
result = model.transform(testData)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

Test set accuracy = 0.7584175084175084


### LinearSVC

In [212]:
from pyspark.ml.classification import LinearSVC

In [219]:
lsvc = LinearSVC(maxIter=10, regParam=0.1)

pipeline = Pipeline(stages=[assembler, LinearSVC])

# Fit the model
lsvcModel = pipeline.fit(trainingData_)

TypeError: Cannot recognize a pipeline stage of type <class 'abc.ABCMeta'>.

In [ ]:
lsvcResult = lsvcModel.transform(testData)

In [ ]:
# Print the coefficients and intercept for linear SVC
print("Coefficients: " + str(lsvcModel.coefficients))
print("Intercept: " + str(lsvcModel.intercept))

AttributeError: 'DataFrame' object has no attribute 'col'